In [173]:
from importlib import reload

import numpy as np

from ics.cobraControl import ethernet
from ics.cobraControl import func, cmds
from ics.cobraControl import convert


class PFI(object):
    CW = 1
    CCW = -1
    DISABLE = 0
    
    nCobrasPerModule = 57
    nModules = 42
    
    def __init__(self, fpgaHost='localhost', doConnect=True, doLoadModel=True):
        self.fpgaHost = fpgaHost
        if doConnect:
            self.connect()
        if doLoadModel:
            self.loadModel()
        
    def connect(self):
        ethernet.sock.connect(self.fpgaHost, 4001)
        
    def disconnect(self):
        ethernet.sock.close()
        ethernet.sock = ethernet.Sock()
        
    def loadModel(self, filename=None):
        """ Load a motormap XML file. """
        
        import ics.cobraOps.CobrasCalibrationProduct as cobraCalib
        import ics.cobraOps.MotorMapGroup as cobraMotorMap
        reload(cobraCalib)
        reload(cobraMotorMap)


        if filename is None:
            filename = "/Users/cloomis/Sumire/PFS/git/ics_cobraOps/python/ics/cobraOps/usedXMLFile.xml"
        
        self.calibModel = cobraCalib.CobrasCalibrationProduct(filename)
        self.motorMap = cobraMotorMap.MotorMapGroup(self.calibModel.nCobras)
        
        self.motorMap.useCalibrationProduct(self.calibModel)
        
    def _freqToPeriod(self, freq):
        """ Convert frequency to 60ns ticks """        
        return int(round(16e3/freq))

    def _periodToFreq(self, freq):
        """ Convert 60ns ticks to a frequency """
        return  (16e3 / per) if (per>=1) else 0

    def _mapCobraIndex(self, cobra):
        """ Convert our module + cobra to global cobra index for the calibration product. """
        
        return ((cobra.module - 1)*self.nCobrasPerModule + cobra.cobraNum-1)
    
    def reset(self, sectors=0x3f):
        err = func.RST(sectors)
        
    def powerCycle(self, sectors=0x3f):
        err = func.POW(sectors)

    def setFreq(self, cobras):
        for c in cobras:
            cobraIdx = self._mapCobraIndex(c)
            thetaPer = self._freqToPeriod(self.calibModel.motorFreq1[cobraIdx]/1000)
            phiPer = self._freqToPeriod(self.calibModel.motorFreq2[cobraIdx]/1000)

            # print(f'set {c.board},{c.cobra} to {thetaPer},{phiPer} {self.calibModel.motorFreq1[c.cobra]}')
            c.p = func.SetParams(p0=thetaPer, p1=phiPer, en=(True, True))
        err = func.SET(cobras)
        
    def moveAllThetaPhi(self, cobras, thetaMove, phiMove, phiHome='ccw'):
        nCobras = self.calibModel.nCobras
        
        phiHomes = np.zeros(nCobras)
        phiMoves = np.zeros(nCobras) + phiMove
        thetaMoves = np.zeros(nCobras) + thetaMove

        thetaSteps, phiSteps = self.motorMap.calculateSteps(thetaMoves, phiHomes, phiMoves)
        
        print('thetaSteps: ', thetaSteps)
        print('phiSteps: ', phiSteps)

        stepMoves = list(zip(thetaSteps.tolist(), phiSteps.tolist()))
        
        cIdx = [self._mapCobraIndex(c) for c in cobras]
        cSteps = [stepMoves[i] for i in cIdx]
        
        self.moveSteps(cobras, cSteps, [('cw', 'cw')]*len(cIdx))
    
    def moveAllSteps(self, cobras, steps, dirs):
        allSteps = [steps]*len(cobras)
        allDirs = [dirs]*len(cobras)
        
        self.moveSteps(cobras, allSteps, allDirs)
        
    def moveSteps(self, cobras, steps, dirs, waitTimes=None):
        
        if len(cobras) != len(steps):
            raise RuntimeError("number of steps must match number of cobras")
        if len(cobras) != len(dirs):
            raise RuntimeError("number of directions must match number of cobras")
        if waitTimes is not None and len(cobras) != len(waitTimes):
            raise RuntimeError("number of waitTimes must match number of cobras")
            
        model = self.calibModel
        
        for c_i, c in enumerate(cobras):
            steps1 = int(steps[c_i][0]), int(steps[c_i][1])
            dirs1 = dirs[c_i]
            en = (steps1[0] != 0, steps1[1] != 0)
            cobraId = self._mapCobraIndex(c)
            
            if dirs1[0] == 'cw':
                ontime1 = model.motorOntimeFwd1[cobraId]
                offtime1 = model.motorOfftimeFwd1[cobraId]
            elif dirs1[0] == 'ccw':
                ontime1 = model.motorOntimeRev1[cobraId]
                offtime1 = model.motorOfftimeRev1[cobraId]
            else:
                raise ValueError(f'invalid direction: {dirs1[0]}')
                
            if dirs1[1] == 'cw':
                ontime2 = model.motorOntimeFwd2[cobraId]
                offtime2 = model.motorOfftimeFwd2[cobraId]
            elif dirs1[1] == 'ccw':
                ontime2 = model.motorOntimeRev2[cobraId]
                offtime2 = model.motorOfftimeRev2[cobraId]
            else:
                raise ValueError(f'invalid direction: {dirs1[1]}')
                
            # For early-late offsets.
            if waitTimes is not None:
                offtime1 = waitTimes[c_i][0]
                offtime2 = waitTimes[c_i][1]
            else:
                offtime1 = offtime2 = 0
                
            c.p = func.RunParams(pu=(int(1000*ontime1), int(1000*ontime2)),
                                 st=(steps1),
                                 sl=(int(1000*offtime1), int(1000*offtime2)),
                                 en=en,
                                 dir=dirs1)
        err = func.RUN(cobras)
        
    def homePhi(self, cobras, nsteps=3000, dir='ccw'):
        steps = [(0,nsteps)]*len(cobras)
        dirs = [(dir,dir)]*len(cobras)
        self.moveSteps(cobras, steps, dirs)
    
    def homeTheta(self, cobras, nsteps=6000, dir='ccw'):
        steps = [(nsteps,0)]*len(cobras)
        dirs = [(dir,dir)]*len(cobras)
        pfi.moveSteps(cobras, steps, dirs)
        
    def cobraBySerial(self, serial):
        """ Find a cobra from its serial number. """
        idx = np.where(self.calibModel.serialIds == serial)
        if len(idx) == 0:
            return None
        return func.Cobra(self.calibModel.moduleIds[idx],
                          self.calibModel.positionerIds[idx])
        
    @classmethod
    def allocateAllCobras(cls):
        return allocateCobraRange(range(1,cls.nModules))

    @classmethod
    def allocateCobraRange(cls, modules, cobraNums=None):
        """ Utility to allocate swaths of cobras:
    
        Args:
          modules (int array-like): a list of 1-indexed boards to allocate from.
          cobras  (int array-like): a list of 1-indexed cobras to allocate from.

        Return:
          cobras
        """
        cobras = []
    
        if np.isscalar(modules):
            modules = [modules]
        for m in modules:
            if m == 0:
                raise IndexError('module numbers are 1-indexed, grrr.')
            if cobraNums is None:
                _cobraNums = range(1,cls.nCobrasPerModule+1)
            else:
                _cobraNums = cobraNums
            
            for c in _cobraNums:
                if c == 0:
                    raise IndexError('cobra numbers are 1-indexed, grrr.')

                cobras.append(func.Cobra(m, c))
        
        return cobras

    @classmethod
    def allocateCobraList(cls, cobraIds):
        cobras = []
        for mc in cobraIds:
            m, c = mc
            if m == 0:
                raise IndexError('module numbers are 1-indexed, grrr.')
            if c == 0:
                raise IndexError('cobra numbers are 1-indexed, grrr.')

            cobras.append(func.Cobra(m, c))
        
        return cobras

    def homePhi(self, cobras, nsteps=3000, dir='ccw'):
        steps = [(0,nsteps)]*len(cobras)
        dirs = [(dir,dir)]*len(cobras)
        self.moveSteps(cobras, steps, dirs)
    
    def homeTheta(self, cobras, nsteps=6000, dir='ccw'):
        steps = [(nsteps,0)]*len(cobras)
        dirs = [(dir,dir)]*len(cobras)
        pfi.moveSteps(cobras, steps, dirs)

In [121]:
# allCobras = PFI.allocateCobraRange(range(1,43))
mod1Cobras = PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = PFI.allocateCobraList([(1,30)])    

moduleCobras = {}
for group in 1,2,3:
    cm = list(range(group,58,3))
    mod = list([1]*len(cm))
    
    moduleCobras[group] = PFI.allocateCobraList(list(zip(mod,cm)))
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

In [122]:
cobras = group3Cobras
# cobras = PFI.allocateCobraList([(1,40)])

In [133]:
print([f"{c.module},{c.cobraNum}" for c in allCobras])

['1,1', '1,2', '1,3', '1,4', '1,5', '1,6', '1,7', '1,8', '1,9', '1,10', '1,11', '1,12', '1,13', '1,14', '1,15', '1,16', '1,17', '1,18', '1,19', '1,20', '1,21', '1,22', '1,23', '1,24', '1,25', '1,26', '1,27', '1,28', '1,29', '1,30', '1,31', '1,32', '1,33', '1,34', '1,35', '1,36', '1,37', '1,38', '1,39', '1,40', '1,41', '1,42', '1,43', '1,44', '1,45', '1,46', '1,47', '1,48', '1,49', '1,50', '1,51', '1,52', '1,53', '1,54', '1,55', '1,56', '1,57']


In [186]:
pfi = PFI(fpgaHost='fpga')
pfi.setFreq(allCobras)

In [187]:
for cobraGroup in moduleCobras.values():
    pfi.homeTheta(cobraGroup, dir='ccw')

In [188]:
def deg2rad(deg):
    return np.pi * deg/180.0

for cobraGroup in moduleCobras.values():
    pfi.moveAllThetaPhi(cobraGroup, deg2rad(180), 0.0)


thetaSteps:  [ 3841.65426635  4529.34149189  2788.09604353  2934.93913279  1596.85032826
  3717.67417599  3435.33316828  3164.5456394   3867.03365859  3038.54171353
  3794.26874936  3138.06907218  2136.23255246  2498.64383634  3693.37044956
  2490.6753835   1610.37843444  3340.22686369  3057.66906186  1890.00600453
  2299.50494443  3184.42502527  2828.79031518  3908.3678931   5241.41592006
  6855.90337167  2651.33481586  2903.35468956  1516.51723331  3084.28124482
  2248.19773112  3804.87573417  2375.65705775  2620.62296195  4705.80503487
  2329.71819593  2906.51352392  2620.62424153  3490.94590567  3946.80032855
  3952.36541923  9800.55023031  3027.05967889  1434.67575495  3773.20365867
  3023.48645433  3159.19067932  3404.48412768  2861.40102733  2174.4175754
  4950.2578749   2487.7198847   2861.88377829  3927.12962785  2452.71558735
  3336.86992897  2346.43951791]
phiSteps:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [179]:
c1 = [pfi.cobraBySerial(2358)]

In [132]:
pfi.homeTheta(cobras, dir='ccw')

In [165]:
pfi.moveAllSteps(allCobras, (1000,0), dirs=('cw','ccw'))

In [76]:
pfi.powerCycle()

In [12]:
print(pfi.calibModel.motorFreq1, pfi.calibModel.motorFreq2)
print(pfi.calibModel.motorOntimeFwd1, pfi.calibModel.motorOntimeFwd2)

[ 63241.  63241.  63745.  63492.  63241.  63492.  62500.  63492.  64000.
  63492.  62992.  63241.  62992.  63492.  64000.  63492.  63492.  62992.
  63241.  64000.  63745.  62992.  63492.  64000.  63241.  62500.  63241.
  63241.  63492.  63492.  63745.  63745.  63745.  63241.  63241.  64000.
  62745.  64000.  62745.  62992.  63492.  62500.  63241.  62256.  64000.
  63492.  64000.  62992.  63492.  64516.  64000.  64257.  64516.  63745.
  64000.  64777.  63492.] [ 108108.  104575.  103896.  104575.  105263.  105960.  105960.  105263.
  104575.  105960.  107382.  105263.  105960.  105263.  105960.  103896.
  105263.  106666.  105263.  105960.  105960.  104575.  104575.  105263.
  105960.  106666.  104575.  108108.  105960.  105263.  108843.  106666.
  105263.  105263.  106666.  105263.  105960.  105263.  104575.  106666.
  105960.  105263.  105960.  104575.  105960.  105263.  105960.  105263.
  106666.  106666.  105960.  105960.  104575.  104575.  103896.  107382.
  103896.]
[ 0.04572   0.

In [13]:
pfi.reset()

NameError: name 'sec_rst' is not defined